In [1]:
import sys
sys.path.append('../harmonizome')

import pandas as pd
import numpy as np
import utility_functions as uf
import timeit
import itertools
import scipy.spatial.distance as dist

In [6]:
%load_ext autoreload
%load_ext memory_profiler
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


### removeAndInpute

In [ ]:
df = pd.DataFrame(data=np.random.randint(10,size=(5, 10)))
df = df.mask(np.random.random(df.shape) < .1)
df.head()

In [ ]:
df.shape

## makeBinaryMatrix

In [ ]:
df = pd.DataFrame(data=np.random.randint(3000,size=(7500, 2)), columns=['Gene Symbol', 'Attributes'])
df = df.set_index('Gene Symbol')
df.head()

In [ ]:
%mprun -f uf.createBinaryMatrix binary_matrix = uf.createBinaryMatrix(df)

In [ ]:
%%memit
binary_matrix.shape

In [ ]:
%mprun -f uf.createSimilarityMatrix similarityuf = uf.createSimilarityMatrix(binary_matrix, 'jaccard', sparse=True)

In [ ]:
%mprun -f uf.createSimilarityMatrix similarity_matrix = uf.createSimilarityMatrix(binary_matrix, 'jaccard')

In [ ]:
np.allclose(similarityuf, similarity_matrix)

In [ ]:
df = pd.DataFrame(data=np.random.randint(3000,size=(10000, 10000)))
np.log2(df + 1)

In [ ]:
uf.zscore(df)

In [3]:
df = pd.DataFrame(data=np.random.randint(300,size=(100, 100)))
df.index.name='Gene Symbol'
df.columns.name='Cell Line'
df.head()

Cell Line,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
Gene Symbol,,,,,,,,,,,,,,,,,,,,,
0,298,259,68,129,262,232,291,82,116,266,...,133,270,151,292,267,78,233,146,44,140
1,237,51,159,240,198,258,225,159,27,227,...,25,130,60,281,284,27,36,65,36,268
2,38,138,227,18,5,72,62,282,284,26,...,74,252,77,196,299,279,299,163,53,130
3,99,124,80,201,129,100,75,125,193,1,...,168,141,232,288,279,263,14,27,36,142
4,180,52,257,61,34,299,213,73,117,125,...,188,294,135,98,278,69,245,179,218,268


In [8]:
uf.createGeneAttributeEdgeList(df)

hello
The number of statisticaly relevent gene-attribute associations is: 9982


Gene Symbol  Cell Line
0            0            298
             1            259
             2             68
             3            129
             4            262
                         ... 
99           95           198
             96            57
             97           149
             98           268
             99            72
Name: Weight, Length: 10000, dtype: category
Categories (300, int64): [0, 1, 2, 3, ..., 296, 297, 298, 299]

In [ ]:
uf.